In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, expr

In [2]:
spark = SparkSession.builder.appName("ch01").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/05 23:13:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/05 23:13:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
schema = '''`CallNumber` INT, 
`UnitID` STRING, 
`IncidentNumber` INT, 
`CallType` STRING, 
`CallDate` STRING, 
`WatchDate` STRING, 
`CallFinalDisposition` STRING, 
`AvailableDtTm` STRING, 
`Address` STRING, 
`City` STRING, 
`Zipcode` INT, 
`Battalion` STRING, 
`StationArea` STRING, 
`Box` STRING, 
`OriginalPriority` STRING, 
`Priority` STRING, 
`FinalPriority` INT, 
`ALSUnit` BOOLEAN, 
`CallTypeGroup` STRING, 
`NumAlarms` INT, 
`UnitType` STRING, 
`UnitSequenceInCallDispatch` INT, 
`FirePreventionDistrict` STRING, 
`SupervisorDistrict` STRING, 
`Neighborhood` STRING, 
`Location` STRING, 
`RowID` STRING, 
`Delay` FLOAT'''

In [ ]:
sf_fire_file = "./learning-spark-v2/sf-fire/sf-fire-calls.csv"
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)